In [1]:
import mysql.connector as sql
import pandas as pd
import datetime
import uuid


In [27]:
def convert_date_col(date_str):
    date_str = date_str[0:16]
    try:
        return datetime.datetime.strptime(date_str, '%m/%d/%Y %H:%M').strftime('%Y-%m-%dT%H:%M')
    except:
        return datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M').strftime('%Y-%m-%dT%H:%M')

In [28]:
db_connection = sql.connect(host='localhost', database='benedict', user='root', password='artema2')

In [29]:
patient_list = pd.read_csv('dka_patients.csv')
patients = list(patient_list['patient_id'])
patients[:5]

['6b129580-229a-11ea-af9c-8a315723aae9',
 '6b5b905a-229a-11ea-af9c-8a315723aae9',
 '6aeaa7a0-229a-11ea-af9c-8a315723aae9',
 '6b046afa-229a-11ea-af9c-8a315723aae9',
 '6b3f03e0-229a-11ea-af9c-8a315723aae9']

In [30]:
qry = "SELECT uuid() AS eventId, patient_id AS patientId, event_date  as eventDate, event_name AS eventName, result_val as resultValue, result_unit AS resultUnit, event_category AS eventCategory "
qry = qry + "FROM benedict.dka_case_events e JOIN benedict.dka_patients p ON e.fin = p.fin "
qry = qry + "WHERE patient_id IN (" + str(patients[:5]).replace('[', '').replace(']', '') + ") "
qry = qry + " ORDER BY patient_id, eventDate "
#qry = qry + "LIMIT 1000;"

print(qry)


SELECT uuid() AS eventId, patient_id AS patientId, event_date  as eventDate, event_name AS eventName, result_val as resultValue, result_unit AS resultUnit, event_category AS eventCategory FROM benedict.dka_case_events e JOIN benedict.dka_patients p ON e.fin = p.fin WHERE patient_id IN ('6b129580-229a-11ea-af9c-8a315723aae9', '6b5b905a-229a-11ea-af9c-8a315723aae9', '6aeaa7a0-229a-11ea-af9c-8a315723aae9', '6b046afa-229a-11ea-af9c-8a315723aae9', '6b3f03e0-229a-11ea-af9c-8a315723aae9')  ORDER BY patient_id, eventDate 


In [31]:
df = pd.read_sql(qry, con=db_connection)

In [32]:
df.shape

(906, 7)

In [33]:
df.head()

,eventId,patientId,eventDate,eventName,resultValue,resultUnit,eventCategory
0,dd2d9a42-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9,2013-08-06 00:20:00,Normal saline,1000.0,mL,bolus
1,dd2d9a7e-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9,2013-08-06 00:36:00,Diastolic BP,54.0,mmHg,vitals
2,dd2d9a92-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9,2013-08-06 00:36:00,Diastolic BP,54.0,mmHg,vitals
3,dd2d9a9c-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9,2013-08-06 00:36:00,Systolic BP,96.0,mmHg,vitals
4,dd2d9a9d-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9,2013-08-06 00:36:00,Respiratory Rate,22.0,RPM,vitals


In [48]:
df['actionType'] = df['eventCategory']
df['actionType'] = df['actionType'].map({'vitals': 'response', 'labs': 'self', 'meds':'trigger', 'bolus':'trigger'})
df['actionType'].value_counts()

response    693
self        161
trigger      52
Name: actionType, dtype: int64

In [49]:
df.shape

(906, 8)

In [50]:
#df['eventDate']

In [51]:
#df['eventDate'] = df['eventDate'].apply(lambda x: convert_date_col(x))
#df['eventDate'] = df['eventDate'].apply(convert_date_col)

In [52]:
df_pat = df['patientId'].value_counts().rename_axis('patientId').reset_index(name='counts')
#df_pat['patientId'] = [uuid.uuid4() for _ in range(len(df_pat.index))]
df_pat.head()

,patientId,counts
0,6aeaa7a0-229a-11ea-af9c-8a315723aae9,282
1,6b046afa-229a-11ea-af9c-8a315723aae9,200
2,6b129580-229a-11ea-af9c-8a315723aae9,149
3,6b3f03e0-229a-11ea-af9c-8a315723aae9,144
4,6b5b905a-229a-11ea-af9c-8a315723aae9,131


In [53]:
df_pat.shape

(5, 2)

In [54]:
df.to_csv('output/dka_events_sorted.csv')
df_pat.to_csv('output/dka_events_patients.csv')

In [55]:
events = []
prev_event_id = ''
prev_pat_id = ''

for idx, row in df.iterrows():
    if prev_pat_id == row['patientId']:
        cur_event = [prev_event_id, row['eventId'], row['patientId']]
        events.append(cur_event)
    
    prev_event_id = row['eventId']
    prev_pat_id = row['patientId']
    
df_linked_events = pd.DataFrame(events)
df_linked_events.columns = ['prevEventId', 'eventId', 'patientId']
df_linked_events.head()

,prevEventId,eventId,patientId
0,dd2d9a42-73ae-11ea-9037-d077e516bc70,dd2d9a7e-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9
1,dd2d9a7e-73ae-11ea-9037-d077e516bc70,dd2d9a92-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9
2,dd2d9a92-73ae-11ea-9037-d077e516bc70,dd2d9a9c-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9
3,dd2d9a9c-73ae-11ea-9037-d077e516bc70,dd2d9a9d-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9
4,dd2d9a9d-73ae-11ea-9037-d077e516bc70,dd2d9aa6-73ae-11ea-9037-d077e516bc70,6aeaa7a0-229a-11ea-af9c-8a315723aae9


In [56]:
df_linked_events.to_csv('output/dka_linked_events.csv')